# Utilisation de vectorize et guvectorize

Numba fournie deux autres fonctionalités (vectorize et guvectorize) permettant d'accélérer des fonctions universelles NumPy (ufuncs). Avant d'aller plus loin, rappelons ce qu'est une ufunc.

Une ufunc est une fonction qui agit élément par élément pour un tableau donné. Elle permet de vectoriser les opérations et elle est définie de la manière suivante: les paramètres d'entrée sont des scalaires et le paramètre de sortie est également un scalaire. 

Prenons par exemple la fonction heaviside qui est définie de la manière suivante

$$
heaviside(x, h_0) = 
\left\{
\begin{array}{l}
0 \; \text{si} \; x<0, \\
h_0 \; \text{si} \; x=0, \\
1 \; \text{si} \; x>0.
\end{array}
\right.
$$

Les ufuncs sont codées en C dans NumPy et sont donc optimisées. Voici la version de la fonction heaviside dans NumPy (attention, cette fonction n'est disponible qu'à partir de la dernière version: 1.13).

```C
/**begin repeat
 * #type = npy_float, npy_double, npy_longdouble#
 * #c = f, ,l#
 * #C = F, ,L#
 */

@type@ npy_heaviside@c@(@type@ x, @type@ h0)
{
    if (npy_isnan(x)) {
        return (@type@) NPY_NAN;
    }
    else if (x == 0) {
        return h0;
    }
    else if (x < 0) {
        return (@type@) 0.0;
    }
    else {
        return (@type@) 1.0;
    }
}
```

Comme vous pouvez le voir ce n'est pas très lisible et surtout, vous n'avez certainement pas envie de faire une interface C pour une fonction simple. (voir si on peut prendre un autre exemple)

**vectorize** et **guvectorize** sont donc là pour vous aider à construire des ufuncs tout en restant en Python.

## vectorize

Voici comment la fonction heaviside s'écrit en Numba en utilisant **vectorize**.

In [1]:
from numba import vectorize

@vectorize(['float64(float64, float64)'])
def heaviside(x, h0):
    if x == 0:
        return h0
    elif x < 0:
        return 0.
    else:
        return 1.

In [2]:
import numpy as np

x = -1 + 2*np.random.random(1000000)
h0 = .4 
numba_res = heaviside(x, h0)
numpy_res = np.heaviside(x, h0)

In [3]:
np.all(numba_res == numpy_res)

True

In [4]:
%timeit heaviside(x, h0)

10 loops, best of 3: 20.1 ms per loop


In [5]:
%timeit np.heaviside(x, h0)

100 loops, best of 3: 8.04 ms per loop


La fonction Numba est 2.5 fois plus lente que la fonction NumPy. On peut accélérer un peu les choses en spécifiant la cible. En effet, **vectorize** a un mot clé **target** indiquant

- 'cpu'

fonction pour un thread sur CPU

- 'parallel'

fonction multi-threads pour CPU

- 'cuda'

fonction utilisant cuda sur GPU

Voyons ce que ça donne si nous mettons cette fonction en parallèle.

In [6]:
@vectorize(['float64(float64, float64)'], target='parallel')
def heaviside_para(x, h0):
    if x == 0:
        return h0
    elif x < 0:
        return 0.
    else:
        return 1.

On l'appelle une première fois pour ne pas prendre en compte le temps de compilation dans le calcul de la performance.

In [7]:
numba_res_para = heaviside_para(x, h0)
np.all(numba_res_para == numpy_res)

True

In [8]:
%timeit heaviside_para(x, h0)

100 loops, best of 3: 7.75 ms per loop


Nous sommes cette fois un peu plus rapide que la fonction NumPy. Mais il faut noté que la version NumPy n'est pas parallèle.

In [9]:
@vectorize(['float64(float64, float64)'], target='cuda')
def heaviside_gpu(x, h0):
    if x == 0:
        return h0
    elif x < 0:
        return 0.
    else:
        return 1.

In [10]:
heaviside_gpu(x, h0)

CudaSupportError: Error at driver init: 
[999] Call to cuInit results in CUDA_ERROR_UNKNOWN:

In [11]:
%timeit heaviside_gpu(x, h0)

100 loops, best of 3: 8.27 ms per loop


Comme vu lors l'introduction à **@jit**, il est possible de spécialiser une fonction vectorize en donnant les différents types sous forme de liste.

In [12]:
@vectorize(['float64(float64, float64)',
            'float32(float32, float32)',
            'int32(int32, int32)'], target='parallel')
def heaviside_para(x, h0):
    if x == 0:
        return h0
    elif x < 0:
        return 0.
    else:
        return 1.

## guvectorize

**guvectorize** fait exactement la même chose que **vectorize** à la seule différence que nous n'avons à aucun moment spécifié le tableau de sortie. Celui-ci est donc créé à chaque appel de la fonction. Ce qui peut avoir un coup non négligeable si nous appelons plein de fois la fonction. **guvectorize** permet de mettre le tableau de sortie dans les arguments.

Reprenons notre exemple

In [90]:
from numba import guvectorize, njit

@njit
def heaviside_core1(x, h0, y):
    for i in range(x.size):
        if x[i] == 0:
            y[i] = h0
        elif x[i] < 0:
            y[i] = 0.
        else:
            y[i] = 1.        

@njit
def heaviside_core(x, h0, y):
    if x == 0:
        y = h0
    elif x < 0:
        y = 0.
    else:
        y = 1.        

@guvectorize(['void(float64[:], float64[:], float64[:])'], '(),()->()', target='parallel')
def guheaviside(x, h0, y):
    heaviside_core(x[0], h0[0], y[0])

In [91]:
y = np.zeros_like(x)
heaviside_core1(x, h0, y)
np.all(y == numpy_res)

True

In [92]:
y

array([ 0.,  0.,  0., ...,  0.,  1.,  1.])

In [93]:
numpy_res

array([ 0.,  0.,  0., ...,  0.,  1.,  1.])

In [94]:
%timeit heaviside_core1(x, h0, y)

100 loops, best of 3: 2.77 ms per loop


In [51]:
from numba import njit

@njit
def splint(xa, ya, y2a, x, y):
    n = xa.shape[0]
    for i in range(x.shape[0]):
        klo = 0
        khi = n-1
        while(khi-klo) > 1:
            k = (khi+klo) >> 1
            if xa[k] > x[i]:
                khi = k
            else:
                klo = k
        h = xa[khi] - xa[klo]
        a = (xa[khi]-x[i])/h    
        b = (x[i]-xa[klo])/h
        y[i,:] = a*ya[klo,:]+b*ya[khi,:]+((a**3-a)*y2a[klo,:]+(b**3-b)*y2a[khi,:])*h**2/6.

In [16]:
from math import sqrt
from numba import njit, jit, guvectorize
import timeit
import numpy as np

@njit
def square_sum(arr):
    a = 0.
    for i in range(arr.size):
        a = sqrt(a**2 + arr[i]**2)  # sqrt and square are cpu-intensive!
    return a

@guvectorize(["void(float64[:], float64[:])"], "(n) -> ()", target="parallel", nopython=True)
def row_sum_gu(input, output) :
    output[0] = square_sum(input)

@jit(nopython=True)
def row_sum_jit(input_array, output_array) :
    m, n = input_array.shape
    for i in range(m) :
        output_array[i] = square_sum(input_array[i,:])
    return output_array


In [17]:
rows = int(64)
columns = int(1e6)

input_array = np.random.random((rows, columns))
output_array = np.zeros((rows))
output_array2 = np.zeros((rows))

# Warmup an check that they are equal 
np.testing.assert_equal(row_sum_jit(input_array, output_array), row_sum_gu(input_array, output_array2))
%timeit row_sum_jit(input_array, output_array.copy())  # 10 loops, best of 3: 130 ms per loop
%timeit row_sum_gu(input_array, output_array.copy())   # 10 loops, best of 3: 35.7 ms per loop

1 loop, best of 3: 2.09 s per loop
1 loop, best of 3: 663 ms per loop


In [27]:
@njit
def square(arr_in, arr_out):
    for i in range(arr_in.size):
        arr_out[i] = sqrt(arr_in[i])
            
@guvectorize(['void(float64[:], float64[:])'], "(n) -> (n)", target="parallel", nopython=True)
def test(input, output):
    square(input, output)
    
@njit
def test_jit(input, output):
    for i in range(input.shape[0]):
        square(input[i], output[i])

In [31]:
n = 10000
input_array = np.random.random((n, n))
output_array = np.zeros((n, n))
output_array2 = np.zeros((n ,n))

In [34]:
%timeit test(input_array, output_array)

1 loop, best of 3: 243 ms per loop


In [35]:
%timeit test_jit(input_array, output_array)

1 loop, best of 3: 761 ms per loop


In [174]:
@jit
def splint_core(xa, ya, y2a, x, y):
    klo = 0
    khi = xa.shape[0]-1
    while(khi-klo) > 1:
        k = (khi+klo) >> 1
        if xa[k] > x:
            khi = k
    #    else:
    #        klo = k
    h = xa[khi] - xa[klo]
    a = (xa[khi]-x)/h    
    b = (x-xa[klo])/h
    y = a*ya[klo]+b*ya[khi]+((a**3-a)*y2a[klo]+(b**3-b)*y2a[khi])*h**2/6.

@guvectorize(['void(float64[:], float64[:], float64[:], float64[:], float64[:])'], "(n),(n),(n),() -> ()", target="parallel")    
def splint_gu(xa, ya, y2a, x, y):
    klo = 0
    khi = xa.shape[0]-1
    while(khi-klo) > 1:
        k = (khi+klo) >> 1
        if xa[k] > x[0]:
            khi = k
        else:
            klo = k
    h = xa[khi] - xa[klo]
    a = (xa[khi]-x[0])/h    
    b = (x[0]-xa[klo])/h
    y[0] = a*ya[klo]+b*ya[khi]+((a**3-a)*y2a[klo]+(b**3-b)*y2a[khi])*h**2/6.
    
@njit
def splint_jit(xa, ya, y2a, x, y):
    n = xa.shape[0]
    for i in range(x.shape[0]):
        klo = 0
        khi = n-1
        while(khi-klo) > 1:
            k = (khi+klo) >> 1
            if xa[k] > x[i]:
                khi = k
            else:
                klo = k
        h = xa[khi] - xa[klo]
        a = (xa[khi]-x[i])/h    
        b = (x[i]-xa[klo])/h
        y[i] = a*ya[klo]+b*ya[khi]+((a**3-a)*y2a[klo]+(b**3-b)*y2a[khi])*h**2/6.

In [169]:
xa = np.linspace(0, 1, 100)
ya = np.random.rand(100)
y2a = np.random.rand(100)

x = np.linspace(0, 1, 1000)
y1 = np.zeros_like(x)
y2 = np.zeros_like(x)

splint_gu(xa, ya, y2a, x, y1)
splint_jit(xa, ya, y2a, x, y2)

In [170]:
%timeit splint_gu(xa, ya, y2a, x, y1)

The slowest run took 114.37 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 64.7 µs per loop


In [171]:
%timeit splint_jit(xa, ya, y2a, x, y2)

10000 loops, best of 3: 135 µs per loop


In [172]:
np.all(y1==y2)

True

In [62]:
import numpy as np
from numba import jit, guvectorize, float64, int64

@jit
def naive_for_loop(some_input_array, another_input_array, result):
    for i in range(result.shape[0]):
        for k in range(some_input_array.shape[0] - i):
            result[i] += some_input_array[k+i] * another_input_array[k] * np.sin(0.001 * (k+i)) 

@guvectorize([(float64[:],float64[:],int64[:],float64[:])],'(n),(n),()->()', nopython=True, target='parallel')
def forall_loop_body_parallel(some_input_array, another_input_array, loop_index, result):
    i = loop_index[0]       # just a shorthand
    # do some nontrivial calculation involving elements from the input arrays and the loop index
    for k in range(some_input_array.shape[0] - i):
        result[0] += some_input_array[k+i] * another_input_array[k] * np.sin(0.001 * (k+i)) 

@guvectorize([(float64[:],float64[:],int64[:],float64[:])],'(n),(n),()->()', nopython=True, target='cpu')
def forall_loop_body_cpu(some_input_array, another_input_array, loop_index, result):
    i = loop_index[0]       # just a shorthand
    # do some nontrivial calculation involving elements from the input arrays and the loop index
    for k in range(some_input_array.shape[0] - i):
        result[0] += some_input_array[k+i] * another_input_array[k] * np.sin(0.001 * (k+i)) 

arg_size = 20000

input_array_1 = np.random.rand(arg_size)
input_array_2 = np.random.rand(arg_size)
result_array = np.zeros_like(input_array_1)

# do single-threaded naive nested for loop
# reset result_array inside %timeit call 
%timeit -r 3 result_array[:] = 0.0; naive_for_loop(input_array_1, input_array_2, result_array)
result_1 = result_array.copy()

# do single-threaded forall loop (loop indices in-order)
# reset result_array inside %timeit call 
loop_indices = range(arg_size)
%timeit -r 3 result_array[:] = 0.0; forall_loop_body_cpu(input_array_1, input_array_2, loop_indices, result_array)
result_2 = result_array.copy()

# do multi-threaded forall loop (loop indices in-order)
# reset result_array inside %timeit call 
loop_indices = range(arg_size)
%timeit -r 3 result_array[:] = 0.0; forall_loop_body_parallel(input_array_1, input_array_2, loop_indices, result_array)
result_3 = result_array.copy()

# do forall loop (loop indices scrambled for better load balancing)
# reset result_array inside %timeit call 
loop_indices_scrambled = np.random.permutation(range(arg_size))
loop_indices_unscrambled = np.argsort(loop_indices_scrambled)
%timeit -r 3 result_array[:] = 0.0; forall_loop_body_parallel(input_array_1, input_array_2, loop_indices_scrambled, result_array)
result_4 = result_array[loop_indices_unscrambled].copy()


# check validity
print(np.all(result_1 == result_2))
print(np.all(result_1 == result_3))
print(np.all(result_1 == result_4))

1 loop, best of 3: 26.9 s per loop
1 loop, best of 3: 26.6 s per loop
1 loop, best of 3: 12.4 s per loop
1 loop, best of 3: 8.98 s per loop
True
True
True


In [18]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom.css").read()
    return HTML(styles)
css_styling()

In [128]:
import math

@vectorize(['float64(float64, float64)'], target='parallel')
def sincos_vec(x, y):
    return math.cos(x)*math.sin(y)

@guvectorize(['(float64[:], float64[:], float64[:])'], '(), () -> ()', target='parallel')
def sincos_gu(x, y, out):
    out[0] = math.cos(x[0])*math.sin(y[0])

@njit
def sincos_jit(x, y, out):
    for i in range(x.size):
         out[i] = math.cos(x[i])*math.sin(y[i])

In [132]:
x = np.linspace(0, 2*math.pi, 1000000)
y = np.linspace(0, 2*math.pi, 1000000)
out_jit = np.zeros_like(x)
out_vec = np.zeros_like(x)
out_gu = np.zeros_like(x)

out_vec = sincos_vec(x, y)

In [133]:
%timeit sincos_vec(x, y)

10 loops, best of 3: 67.1 ms per loop


In [134]:
%timeit sincos_jit(x, y, out_jit)

1 loop, best of 3: 213 ms per loop


In [135]:
%timeit sincos_gu(x, y, out_gu)

10 loops, best of 3: 72.5 ms per loop


In [136]:
print(np.all(out_vec == out_jit))
print(np.all(out_vec == out_gu))

True
True
